# PyTorch Tutorials
## Quickstart
https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

This section runs through API for common tasks in machine learning.
Refer to the links in each section to dive deeper.

このセクションでは，機械学習の一般的なタスクをAPIを通して簡単に実行する．
詳細は各リンクを参考にする．

## Working with data

PyTorch has two primitives to work with data: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset`.
`Dataset` stores the samples and their corresponding labels, and `DataLoader` wraps an itetable around the `Dataset`.

PyTorchは二つのプリミティブにデータを扱う方法がある．

- `torch.utils.data.DataLoader`
    - データセットを反復可能なラップする．
- `torch.utils.data.Dataset`
    - サンプルとそれに対応するラベルを格納する

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

PyTorch offers domain-specific libraries such as [TorchText](https://pytorch.org/text/stable/index.html), 
[TorchVision](https://pytorch.org/vision/stable/index.html), 
[TorchAudio](https://pytorch.org/audio/stable/index.html), all of which include datasets.
For this tutorial, we will be using TorchVision dataset.

PyTorchはTorchText, TorchVision, TorchAudioといったドメイン特有のライブラリを提供している．
このチュートリアルでは，TorchVisionを用いる．

The `torchvision.datasets` module contains `Dataset` objects for many real-world vision data like CIFAR, COCO([full list here](https://pytorch.org/vision/stable/datasets.html)).
In this tutorial, we use the FashionMNIST dataset.
Every TorchVision `Dataset` includes two arguments: `transform` and `target_transform` to modify the samples and labels respectively.

`torchvision.datasets`モジュールには，CIFARやCOCOといった実世界の画像データを持つ`Dataset`オブジェクトが含まれている．
本チュートリアルでは，FashionMNISTデータセットを用いる．
TorchVisionの全ての`Dataset`には，二つの引数`transform`と`target_trainsform`があり，サンプルとラベルを調整することができる．

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Processing...
Done!


/Users/shibanotoshiki/.pyenv/versions/3.8.3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


We pass the `Dataset` as an argument to `DataLoader`.
This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading.
Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

`DataLoader`に`Dataset`を引数として渡す．
これにより，データセットに反復可能性がラップされ，自動的なバッチ処理，サンプリング，シャッフル，マルチプロセスデータ読み込みがサポートされる．
ここでは，バッチサイズを64と定義する．つまり，データローダーイテラブルのそれぞれの要素は64個の特徴量(入力データ？)とラベルをを返す．

In [3]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print('Shape of X[N, C, H, W]', X.shape)
    print('Shape of y: ', y.shape, y.dtype)
    break

Shape of X[N, C, H, W] torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


Read more about [loading data in PyTorch](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

## Creating Models
To define a neural network in PyTorch, we create a class that inherits from [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
We define the layers of the network in the `__init__` function and specify how data will pass through the network in the `forward` function.
To accelerate operations in the neural network, we move it to the GPU if available.

PyTorchにおいて，ニューラルネットワークを定義するには，nn.Moduleを継承するクラスを作成する．
`__init__`関数内でネットワークの層を定義し，`forward`関数内でデータがどのようにネットワークを通るかを記述する．
利用できるなら，速度を上げるためにGPUにネットワークを移す．

In [4]:
# Get cpu or gpu device for training.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device.'.format(device))


Using cpu device.


In [5]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


Read more about building [neural networks in PyTorch](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

## Optimizing the Model Parameters
To train a model, we need a loss function and an optimizer.

モデルを9連するために，損失関数と最適化手法が必要．

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the models's parameters.

一回の訓練ループの中で，モデルは訓練データセットに対する予測をし，予測誤差を逆伝播してモデルのパラメータを調整する

In [7]:
def train(dataloader, model, loss_fn, optimzer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

We also check the model's performance against the test dataset to ensure it is learning.

モデルが学習しているかを確認するために，テストデータセットに対するモデルのパフォーマンスをチェックする

In [8]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

The training process is conducted over several iterations (epochs). 
During each epoch, the model learns parameters to make better predictions.
We print the model's accuracy and loss at each epoch; we'd like to see the accuracy increase and the loss decrease with every epoch.

訓練課程は何回か行う(epoch)．
エポックの間，モデルはより良い予測をするためにパラメータを学習する．
エポックごとに，モデルの精度と誤差を出力する；エポックごとに，精度が増加し，誤差が減少するのが見える．

In [12]:
epochs = 5
torch.manual_seed(0)
for t in range(epochs):
    print(f'Epoch {t+1}\n------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print('Done')

Epoch 1
------------------------
loss: 2.306999 [    0/60000]
loss: 2.306620 [ 6400/60000]
loss: 2.298018 [12800/60000]
loss: 2.296527 [19200/60000]
loss: 2.297415 [25600/60000]
loss: 2.289053 [32000/60000]
loss: 2.281904 [38400/60000]
loss: 2.276269 [44800/60000]
loss: 2.284163 [51200/60000]
loss: 2.271591 [57600/60000]
Test Error: 
 Accuracy: 19.8%, Avg loss: 0.035701 

Epoch 2
------------------------
loss: 2.276786 [    0/60000]
loss: 2.294239 [ 6400/60000]
loss: 2.271836 [12800/60000]
loss: 2.269336 [19200/60000]
loss: 2.274726 [25600/60000]
loss: 2.266478 [32000/60000]
loss: 2.247821 [38400/60000]
loss: 2.240928 [44800/60000]
loss: 2.258650 [51200/60000]
loss: 2.235213 [57600/60000]
Test Error: 
 Accuracy: 20.0%, Avg loss: 0.035165 

Epoch 3
------------------------
loss: 2.250551 [    0/60000]
loss: 2.281789 [ 6400/60000]
loss: 2.240177 [12800/60000]
loss: 2.231298 [19200/60000]
loss: 2.242226 [25600/60000]
loss: 2.237740 [32000/60000]
loss: 2.201281 [38400/60000]
loss: 2.192081

Read more about [Training your model](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)

## Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model performance).

モデルの一般的な保存方法は，内部状態ディクショナリをシリアル化することである．

In [14]:
torch.save(model.state_dict(), 'model.pth')
print('Saved PyTorch Model State to model.pth')

Saved PyTorch Model State to model.pth


## Loading Models
The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

モデルを読み込む過程には，モデル構造の再作成と状態ディクショナリのロードがある．

In [16]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

This model can now be used to make predictions.

このモデルはすぐに予測に使うことができる．

In [19]:
classes = ['T-shart/top',
           'Trouser',
           'Pullover',
           'Dress',
           'Coat',
           'Sandal',
           'Shirt',
           'Sneaker',
           'Bag',
           'Ankle boot'
          ]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predited: {predicted}, Actual: {actual}')

Predited: T-shart/top, Actual: Ankle boot


Read more about [Saving & Loading your model](https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html)